In [2]:
import subprocess
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf

flask_process = subprocess.Popen(['python3', 'apps.py'])

print(f"Flask app is running with PID {flask_process.pid}")

Flask app is running with PID 4682


2024-06-17 23:00:32.529881: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 23:00:33.390320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ubuntu/code/DE_ML/HITIT_Server/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ubuntu/code/DE_ML/HITIT_Server/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to 

Model loaded from ./RF/loaded_model
입력 데이터: [[42454, 40, 32, 29.33, 291]]
['안정적']
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.98365}]
입력 데이터: [[42454, 40, 32, 29.33, 291], [42454, 40, 32, 29.33, 291], [42454, 40, 32, 29.33, 291]]
['안정적', '안정적', '안정적']
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.98365}]
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.98365}]


In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")
model = AutoModelForMaskedLM.from_pretrained("kakaobank/kf-deberta-base")

# Tokenize하기

In [2]:
input_text = "서유준의 매출이 하락했으며 장래가 불투명합니다"
tokens = tokenizer.tokenize(input_text) #['서유','##준','##의','매출','##이','하락','##했', ##으며','장래',...] 이런식으로

inputs = tokenizer(input_text, return_tensors="pt") #카카오 그 모델에 등록된 단어와 매칭시켜서 숫자로 

model_output = model(**inputs)

# [MASK]된 단어 예측하기

In [91]:
# 모델을 사용하여 예측 logits 계산

input_text = "매출이 상승했으며 [MASK]가 유망합니다"
tokens = tokenizer.tokenize(input_text)
print(tokens)

inputs = tokenizer(input_text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

# 토큰의 가장 가능성 높은 예측값 가져오기
predicted_token_id = torch.argmax(logits[0, 3]).item()  # [MASK] 위치의 토큰 (여기서 3번째 위치)
predicted_token = tokenizer.convert_ids_to_tokens(predicted_token_id)

# # [MASK] 토큰을 예측된 토큰으로 대체하여 문장 복원
restored_text = input_text.replace("[MASK]", predicted_token)
print(f"Restored text: {restored_text}")

['매출', '##이', '상승', '##했', '##으며', '[MASK]', '가', '유망', '##합니다']
Restored text: 매출이 상승했으며 상승가 유망합니다


# 감정분석하기

In [3]:
from transformers import pipeline
# 감정 분석 파이프라인 로드
sentiment_model = pipeline(model="WhitePeak/bert-base-cased-Korean-sentiment")

In [4]:
def get_sentiment(text):
    result = sentiment_model(text)
    for elem in result :
        if elem['label'] == "LABEL_1" :
            elem['label'] = "POSITIVE"
        else :
            elem['label'] = "NEGATIVE"
        elem['score'] = round(elem['score'],5)

    print(f"Sentiment analysis result: {result}")
    return result

# 뉴스데이터 크롤링하기

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def naver_news_crawling(keyword_list, day):
    result_object = {}  # 결과를 저장할 객체

    for keyword in keyword_list:
        url = f"https://search.naver.com/search.naver?where=news&query={keyword} \
        &sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={day}&de={day}&docid=&related=0&mynews=0&office_type=0&office_section_code=0 \
        &news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240101to20240101&is_sug_officeid=0&office_category=0&service_area=0"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        result = []
        for item in soup.select('.list_news .bx'):
            press = item.select_one('a.info.press').get_text().replace(' 선정', '').strip()
            anchor = item.select_one('a.news_tit')
            title = anchor.get_text().strip()
            url = anchor['href']
            dsc = item.select_one('.news_dsc').get_text().strip()

            result.append({
                'press': press,
                'title': title,
                'url': url,
                'dsc': dsc,
            })

        result_object[keyword] = result  # 키워드를 키로 사용하여 결과 저장

    return result_object  # 객체 반환

In [8]:
keyword_list = ["삼성전자"]
day = "2024.06.04"
results = naver_news_crawling(keyword_list, day)

#뉴스통해서 받은 제목과 기사내용을 바탕으로 감정분석하기
for keyword, articles in results.items():
    print(f"Keyword: {keyword}")
    for article in articles:
        print(f"Title: {article['title']}")
        print("Title 감정분석 들어갑니다~")
        get_sentiment(article['title'])
        
        print(f"Description: {article['dsc']}")
        print("Desc 감정분석 들어갑니다")
        get_sentiment(article['dsc'])
        
        print("\n------------------------------\n")

Keyword: 삼성전자
Title: 젠슨 황 "삼성 HBM 인증 절차 진행중…테스트 실패한 적 없어"
Title 감정분석 들어갑니다~
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.97087}]
Description: 대만 기자간담회서 언급…"삼성 HBM, 더 많은 엔지니어링 작업 필요" 젠슨 황 엔비디아 최고경영자(CEO)가 삼성전자의 고대역폭 메모리(HBM)와 관련, 인증 테스트에 실패한 적이 없다면서 인증 절차를 진행 중이라고 4일(이하 현지시간) 밝혔다. 블룸버그통신에 따르면 황 CEO는 이날 대만 타이베이 한 호텔에서...
Desc 감정분석 들어갑니다
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.9784}]

------------------------------

Title: 젠슨 황 "삼성전자, 엔비디아 HBM 테스트 실패한 적 없어"(종합2보)
Title 감정분석 들어갑니다~
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.90266}]
Description: '삼성 테스트 실패설' 부인…"HBM 검사 중, 아직 끝나지 않았을 뿐" "하이닉스·삼성·마이크론 모두 메모리 공급"…엔비디아 납품 '3파전' 이봉석 김아람 기자 = 젠슨 황 엔비디아 최고경영자(CEO)가 삼성전자의 고대역폭 메모리(HBM)가 엔비디아 제품에 탑재될 가능성을 시사했다. 그는 삼성전자 HBM이...
Desc 감정분석 들어갑니다
Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.94101}]

------------------------------

Title: "조합원수 뻥튀기에 중복계정까지"…삼성전자 전삼노 비위 공개
Title 감정분석 들어갑니다~
Sentiment analysis result: [{'label': 'NEGATI

In [10]:
# 모바일 내에서 검색한 뉴스들의 제목 & 내용 가져옴
def mobile_news_crawling(keyword_list, day):
    result_object = {}  # 결과를 저장할 객체

    for keyword in keyword_list:
        url = f"https://m.search.naver.com/search.naver?where=m_news&query={keyword}&sm=mtb_opt&sort=0&photo=0&field=0&pd=3 \
        &ds={day}&de={day}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&is_sug_officeid=0 \
        &office_category=0&service_area=0"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        result = []
        for item in soup.select('.list_news .bx'):
            
            press = item.select_one('a.info.press').get_text().replace(' 선정', '').strip()
            anchor = item.select_one('a.news_tit')
    
            title = anchor.get_text().strip()
            url = anchor['href']
            dsc = item.select_one('.news_dsc').get_text().strip()
            result.append({
                'press': press,
                'title': title,
                'url': url,
                'dsc': dsc,
            })

        result_object[keyword] = result  # 키워드를 키로 사용하여 결과 저장
    
    return result_object  # 객체 반환


In [11]:
import requests
#네이버 모바일에서의 뉴스 포맷을 크롤링
def get_mobile_dsc(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    result = []
    for item in soup.find('article', id='dic_area'):
        result.append(item.get_text(strip=True))
    
    return result
    
mobile_result = mobile_news_crawling(['삼성전자'], "2024.06.01")
get_mobile_dsc("https://n.news.naver.com/article/262/0000017491?sid=101")

['',
 '[경제사상가 이건희 탐구㊾] 이승한 전 홈플러스 창업회장 증언 ㊦',
 '',
 '',
 '● 삼성 적극 유치한 테일러시, 29년 전 英 윈야드 닮아● 건설 비용 41% 보조, 부지 헐값 제공한 英 정부● 96년 이건희가 꿈꾼 첨단 전자 복합단지 ‘도곡동 파크’● ‘삼성 특혜’ 여론에 결국 타워팰리스 들어서● 한남동 리움 미술관 건립 초기부터 관여한 이건희● 이건희 회장은 ‘실용주의 사상가’ ‘파괴적 혁신자’',
 '',
 '',
 '',
 '',
 '',
 '이승한 전 홈플러스 창업회장. [조영철 기자]',
 '4월 15일 바이든 미국 행정부는 현재 텍사스주 테일러시에 반도체 공장을 짓고 있는 삼성전자에 8조8640억 원(64억 달러)에 달하는 막대한 보조금을 지원하겠다고 밝혔다. 삼성이 당초 예상했던 규모보다 세 배가 넘는다고 한다.',
 '',
 '',
 '삼성전자는 대규모 투자 계획으로 화답했다. 공장을 추가로 하나 더 짓는 것은 물론 생산, 조립, 연구개발까지 한번에 할 수 있는 ‘반도체 복합화 단지’를 건설한다는 것이다. 테일러 공장에서는 최첨단 공정인 4나노미터(㎚·10억분의 1미터)이하 반도체를 생산할 예정이다. 바이든 행정부는 “전 세계에서 가장 발전된 기술을 가진 삼성이 미 본토에서 생산하게 됐다”며 자국민에게 성과를 자랑하느라 바빴다.',
 '',
 '',
 '이번 발표에서 의미심장했던 또 다른 한 가지는 삼성전자가 미 국방부 등 국방·안보 분야와 관련된 부처들로부터 직접 반도체를 ‘맞춤 수주’ 받아 생산·공급한다는 것이다. 레이얼 브레이너드 백악관 국가경제위원회(NEC) 위원장은 “삼성이 미국 국가안보를 강하게 만들 것”이라고 했다. 가장 예민한 국가안보와 관련한 반도체 칩 제조를 한국에 맡긴 것이다.',
 '',
 '',
 '미국은 삼성뿐 아니라 대만의 세계 최고 파운드리 업체 TSMC에도 비슷한 규모의 천문학적 보조금을 지원할 예정인데 이는 첨단 기업들의 기술 및 설비를 본토로 빨아들이겠다는 큰 그림에서 나온 전략이다. 미

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

In [253]:
def fetch_page(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response
    except requests.RequestException as e:
        print(f"Error fetching page: {url}", e)
        return None

def decode_html(response):
    content_type = response.headers.get("content-type")
    charset = "UTF-8"
    if "charset=" in content_type:
        charset = content_type.split("charset=")[1]
    return response.content.decode(charset)

def fetch_main(url, start_page, end_page):
    response = fetch_page(url)
    if response:
        data = decode_html(response)
        soup = BeautifulSoup(data, 'html.parser')
        title = "뉴스 리스트영역"
        name = "news"; 
        
        news_src = soup.find('iframe', {'title': title, 'name': name})['src']
        news_src = news_src.replace("&page=&clusterId=","")

        results = []
        for page in range(start_page, end_page + 1):
            page_url = f"{base_url}{news_src}&page={page}"
            results.extend(fetch_news(page_url, end_page))
            
        return results
    
#url 하나 당 뉴스 쫘라락 있는데 그거 가져오기
def fetch_news(url, end_page): 
    response = fetch_page(url)
    if response:
        data = decode_html(response)
        soup = BeautifulSoup(data, 'html.parser')
        results = []
        table_rows = soup.select("table.type5 tr")
            
        for row in table_rows:
            cells = row.find_all('td')
            if len(cells) == 3:
                title = cells[0].text.strip()
                link = base_url + cells[0].a['href']
                provider = cells[1].text.strip()
                date = cells[2].text.strip()
                results.append({
                    "title": title,
                    "link": link,
                    "provider": provider,
                    "date": date,
                })
        return results
    return []



1 부터 5 까지 수집합니다..
롯데하이마트온라인쇼핑몰, 에너지효율 1등급 에어컨 기획전 진행
https://finance.naver.com/item/news_read.naver?article_id=0002836939&office_id=119&code=005930&page=1&sm=
롯데하이마트온라인쇼핑몰, ‘에너지효율 1등급 에어컨 기획전’ 진행
https://finance.naver.com/item/news_read.naver?article_id=0005756728&office_id=018&code=005930&page=1&sm=
롯데하이마트, 효율 1등급 에어컨 기획전…최대 80만원 할인
https://finance.naver.com/item/news_read.naver?article_id=0014729082&office_id=001&code=005930&page=1&sm=
롯데하이마트, 전력 사용량 부담에…에너지 고효율 상품 할인전 진행
https://finance.naver.com/item/news_read.naver?article_id=0005195281&office_id=014&code=005930&page=1&sm=
롯데하이마트 온라인쇼핑몰, 에너지효율 '1등급' 에어컨 기획전 진행
https://finance.naver.com/item/news_read.naver?article_id=0012588343&office_id=003&code=005930&page=1&sm=
롯데하이마트, 전력 사용량 부담에…에너지 고효율 상품 할인전 진행
https://finance.naver.com/item/news_read.naver?article_id=0007583219&office_id=421&code=005930&page=1&sm=
롯데하이마트 온라인쇼핑몰, 에너지효율 1등급 에어컨 기획전...최대 ...
https://finance.naver.com/item/news_read.naver?article_id=0003212034&office_id=030&cod

In [ ]:
code = "005930"
url = f"https://finance.naver.com/item/news.nhn?code={code}"

returned_data = fetch_main(url, 1, 5)
for data in returned_data :
    print(data['title'])
    print(data['link'])

In [13]:
import socket

def get_local_ip():
    try:
        hostname = socket.gethostname()
        local_ip = socket.gethostbyname(hostname)
        return local_ip
    except socket.error as e:
        print(f"Error fetching local IP: {e}")
        return None

local_ip = get_local_ip()
print(f"Local IP: {local_ip}")


Local IP: 172.31.13.91


In [14]:
from transformers import pipeline
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
import joblib
# 감정 분석 파이프라인 로드

def get_sentiment(text):
    result = sentiment_model(text)
    for elem in result :
        if elem['label'] == "LABEL_1" :
            elem['label'] = "POSITIVE"
        else :
            elem['label'] = "NEGATIVE"
        elem['score'] = round(elem['score'],5)

    print(f"Sentiment analysis result: {result}")
    return result


app = Flask(__name__)

@app.route('/sentiment', methods=['POST'])
def predict():
    global sentiment_model
    try:
        data = request.json['data']
        sentiment_result = get_sentiment(data)
        
        print(sentiment_result)
        return jsonify({'sentiment analysis': sentiment_result })
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == "__main__":
    global sentiment_model
    sentiment_model = pipeline(model="WhitePeak/bert-base-cased-Korean-sentiment")
    # print(f'Model loaded from {model_path}')

    from waitress import serve
    serve(app, host="0.0.0.0", port=8081)

Sentiment analysis result: [{'label': 'NEGATIVE', 'score': 0.98365}]
[{'label': 'NEGATIVE', 'score': 0.98365}]
